In [5]:
from alpha_vantage.timeseries import TimeSeries
import pandas as pd

def save_dataset(symbol):
    api_key = 'YYYC6JRUF7KDUUZR'  # Recuerda reemplazar esto con tu propia clave de API
    ts = TimeSeries(key=api_key, output_format='pandas')
    
    try:
        data, meta_data = ts.get_daily(symbol, outputsize='full')
        data.to_csv('./{}_daily.csv'.format(symbol))
        print("Datos guardados exitosamente como '{}_daily.csv'".format(symbol))
    except Exception as e:
        print("Error al obtener los datos:", e)

symbol = 'GOOGL'  # El símbolo de Google en Alpha Vantage es 'GOOGL'
save_dataset(symbol)


Datos guardados exitosamente como 'GOOGL_daily.csv'


In [6]:
nombre_archivo = '{}_daily.csv'.format(simbolo)

In [7]:
import pandas as pd
from sklearn import preprocessing
import numpy as np

history_points = 50

def csv_to_dataset(csv_path):
    datos = pd.read_csv(csv_path)

    print('Datos: ', datos.head(),'\n')

    # Elimina el primer día
    datos  = datos.drop(0, axis=0)

    # Ordena de más viejo a  más reciente
    datos = datos.sort_values('date')
    print('Datos por Fecha: ', datos.head(),'\n')

    # Elimina la columna 'date'  
    datos = datos.drop('date', axis=1)

    print("Tipo de de los Datos:", type(datos),'\n')
    print("Datos: ", datos[:10],'\n')


    normalizador = preprocessing.MinMaxScaler()
    datos_norm = normalizador.fit_transform(datos)

    print("Datos normalizados", datos_norm[:10],'\n')
    # Usamos los últimos 50 ejemplos para predecir el siguiente valor (open)
    # X
    ohlcv_histories_normalised =      np.array([datos_norm[i  : i + history_points].copy() for i in range(len(datos_norm) - history_points)])
    
    
    #y
    next_day_open_values_normalised = np.array([datos_norm[:,0][i + history_points].copy() for i in range(len(datos_norm) - history_points)])
    next_day_open_values = np.array([datos.iloc[:,0][i + history_points].copy() for i in range(len(datos) - history_points)])
    next_day_open_values = np.expand_dims(next_day_open_values_normalised, -1)
    print('next_day_open_values shape: ',next_day_open_values.shape)
    print(next_day_open_values[:10], sep = '\n')

    
    y_scaler = preprocessing.MinMaxScaler()
    y_scaler.fit( next_day_open_values  )

    # Indicadores tecnicos
    technical_indicators = []
    for his in ohlcv_histories_normalised:
        # Promedio del Precio de cierre 
        sma = np.mean(his[:,3])
        technical_indicators.append(np.array([sma]))

    technical_indicators = np.array(technical_indicators)
		
    tech_ind_scaler = preprocessing.MinMaxScaler()
    technical_indicators_normalised = tech_ind_scaler.fit_transform(technical_indicators)



    # Verifica que el número de xs sea igual al número de ys
    assert ohlcv_histories_normalised.shape[0] == next_day_open_values_normalised.shape[0] == technical_indicators_normalised.shape[0]

    return ohlcv_histories_normalised, technical_indicators_normalised, next_day_open_values_normalised, next_day_open_values, y_scaler  

In [13]:
import os
print(os.listdir())

ohlcv_histories, technical_indicators, next_day_open_values, unscaled_y, y_scaler = csv_to_dataset('{}_daily.csv'.format(simbolo))

test_split = 0.9
n = int(ohlcv_histories.shape[0] * test_split)

ohlcv_train = ohlcv_histories[:n]
tech_ind_train = technical_indicators[:n]
y_train = next_day_open_values[:n]

ohlcv_test = ohlcv_histories[n:]
tech_ind_test = technical_indicators[n:]
y_test = next_day_open_values[n:]

unscaled_y_test = unscaled_y[n:]



['mapa_contornos.png', 'objetos_azules', 'untitled.md', 'Busqueda_de_planos_a_partir_de_una_nube_de_puntos.ipynb', 'analisisAPPLE.ipynb', 'bynary.ipynb', 'Untitled.ipynb', 'generaExtracionContornos.ipynb', 'contornosTIme6.ipynb', 'conjunto_2.nc', 'generarImagen.ipynb', 'analisisnetcdf.ipynb', 'Untitled6.ipynb', 'procesObjet.py', 'componentConnected.ipynb', 'conjunto_6.nc', 'imagenBIn.py', 'saturation.ipynb', 'nombre_del_archivo.nc', 'objetos_contornos', 'graficarDATOS.ipynb', 'time6', 'analisisTecnico.ipynb', 'conjunto_4.nc', 'conjunto_3.nc', 'objetos_binarizados', 'genContour.ipynb', 'analisiss.rar', 'GENERA_OBJET_BINARIZADO.ipynb', 'genImg1.ipynb', 'prueba.ipynb', 'genNETCDF.ipynb', 'csv2.ipynb', 'Untitled5.ipynb', 'GENERA_COMPONENTES_CONECTADOS.ipynb', 'objetoSinContorno.ipynb', 'generarBinarizado.ipynb', 'conjunto_5.nc', '.ipynb_checkpoints', 'time6.ipynb', 'imagen_coloreada_contornos_cafe.png', 'predicionBolsa.ipynb', 'objetos_componentes_conectados', 'imagen_objetos_azules_contor

FileNotFoundError: [Errno 2] No such file or directory: 'IXIC_daily.csv'

In [ ]:
%tensorflow_version 2.16.1.x

UsageError: Line magic function `%tensorflow_version` not found.


In [11]:
import keras
import tensorflow as tf
from keras.models import Model
from keras.layers import Dense, Dropout, LSTM, Input, Activation, concatenate
from keras import optimizers
import numpy as np



# define two sets of inputs
lstm_input = Input(shape=(history_points, 5), name='lstm_input')
dense_input = Input(shape=(technical_indicators.shape[1],), name='tech_input')
 
# the first branch operates on the first input
x = LSTM(50, name='lstm_0')(lstm_input)
x = Dropout(0.2, name='lstm_dropout_0')(x)
lstm_branch = Model(inputs=lstm_input, outputs=x)
 
# the second branch opreates on the second input
y = Dense(20, name='tech_dense_0')(dense_input)
y = Activation("relu", name='tech_relu_0')(y)
y = Dropout(0.2, name='tech_dropout_0')(y)
technical_indicators_branch = Model(inputs=dense_input, outputs=y)
 
# combine the output of the two branches
combined = concatenate([lstm_branch.output, technical_indicators_branch.output], name='concatenate')
 
z = Dense(64, activation="sigmoid", name='dense_pooling')(combined)
z = Dense(1, activation="linear", name='dense_out')(z)
 
# our model will accept the inputs of the two branches and then output a single value
model = Model(inputs=[lstm_branch.input, technical_indicators_branch.input], outputs=z)

adam = optimizers.Adam(lr=0.0005)

model.compile(optimizer=adam,
              loss='mse')

from keras.utils import plot_model
plot_model(model, to_file='model.png', show_shapes=True)

ModuleNotFoundError: No module named 'keras'

In [ ]:
model.fit(x=[ohlcv_train, tech_ind_train], y=y_train, batch_size=32, epochs=50, shuffle=True, validation_split=0.1)

In [ ]:
<keras.callbacks.History at 0x7f40860de978>

In [ ]:
# Las predicciones estásn normalizadas
y_test_predicted = model.predict([ohlcv_test, tech_ind_test])

y_test_predicted = y_scaler.inverse_transform(y_test_predicted)

real_mse = np.mean(np.square(unscaled_y_test - y_test_predicted))
scaled_mse = real_mse / (np.max(unscaled_y_test) - np.min(unscaled_y_test)) * 100
print(scaled_mse)
     

In [ ]:
import matplotlib.pyplot as plt
plt.gcf().set_size_inches(22, 15, forward=True)

start = 0
end = -1

real = plt.plot(unscaled_y_test[start:end], label='real')
pred = plt.plot(y_test_predicted[start:end], label='predicted')

plt.legend(['Real', 'Predicted'])

plt.show()


In [ ]:
buys = []
sells = []
thresh = 0.01#0.2

x = 0
for ohlcv, ind in zip(ohlcv_test, tech_ind_test):
    normalised_price_today = ohlcv[-1][0]
    normalised_price_today = np.array([[normalised_price_today]])
    price_today = y_scaler.inverse_transform(normalised_price_today)
    predicted = np.squeeze(y_scaler.inverse_transform( model.predict([[ohlcv], [ind]])))
    delta = predicted - price_today
    # print(delta)
    if delta > thresh:
        buys.append((x, price_today[0][0]))
    elif delta < -thresh:
        sells.append((x, price_today[0][0]))
    x += 1
print(len(buys))
print(len(sells))

In [ ]:
import matplotlib.pyplot as plt

plt.gcf().set_size_inches(22, 15, forward=True)

start = 0
end = -1

real = plt.plot(unscaled_y_test[start:end], label='real')
pred = plt.plot(y_test_predicted[start:end], label='predicted')

plt.scatter(list(list(zip(*buys))[0]), list(list(zip(*buys))[1]), c='#00ff00')  # verde
plt.scatter(list(list(zip(*sells))[0]), list(list(zip(*sells))[1]), c='#ff0000') # rojo

# real = plt.plot(unscaled_y[start:end], label='real')
# pred = plt.plot(y_predicted[start:end], label='predicted')

plt.legend(['Real', 'Predicted'])


plt.savefig('predicciones.png')

plt.show()

In [ ]:

def compute_earnings(buys, sells):
    purchase_amt = 10
    stock = 0
    balance = 0
    while len(buys) > 0 and len(sells) > 0:
        if buys[0][0] < sells[0][0]:
            # time to buy $10 worth of stock
            balance -= purchase_amt
            stock += purchase_amt / buys[0][1]
            buys.pop(0)
        else:
            # time to sell all of our stock
            balance += stock * sells[0][1]
            stock = 0
            sells.pop(0)
    print(balance)
     

In [ ]:
compute_earnings(buys, sells)